**Imports**

In [1]:
from deepcheck import *

c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Asus\Desktop\Phosphene.AI\sizeinvar_timesformer\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
save_config(["C:\\Users\\Asus\\Desktop\\Phosphene.AI\\timesformer\\MINTIME-Multi-Identity-size-iNvariant-TIMEsformer-for-Video-Deepfake-Detection\\examples\\fake_1_face_0.mp4"])

C:\Users\Asus\Desktop\Phosphene.AI\timesformer\MINTIME-Multi-Identity-size-iNvariant-TIMEsformer-for-Video-Deepfake-Detection\examples\fake_1_face_0.mp4
Face detection completed.
Cropping faces from the video...
Faces cropping completed.
Clustering faces...
Faces clustering completed.
Searching for fakes in the video...
Features Extractor checkpoint loaded.


c:\Users\Asus\Desktop\Phosphene.AI\DeepCheck\deepcheck.py:228: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  return torch.tensor([sequence]).float(), torch.tensor([size_embeddings]).int(), torch.tensor([mask]).bool(), torch.tensor([identities_mask]).bool(), torch.tensor([positions]), tokens_per_identity


The video is fake (100.0%), showing video result...


['./fake_1_face_0.avi']

In [10]:
import boto3
import os
from botocore.exceptions import NoCredentialsError
from botocore import UNSIGNED
from botocore.config import Config

In [15]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

In [19]:
s3.Bucket("pretrained-baseline-model").download_file("MINTIME_XC_Extractor_checkpoint30", './preprocessing/MINTIME_XC_Extractor_checkpoint30')

In [3]:
bucket_name, object_key, local_file_path = "pretrained-baseline-model" , "MINTIME_XC_Extractor_checkpoint30", "./"

In [4]:
s3.download_file(bucket_name, object_key, local_file_path)

NoCredentialsError: Unable to locate credentials

In [ ]:
import boto3
import os
from botocore.exceptions import NoCredentialsError

def download_from_s3(bucket_name, object_key, local_file_path):
    # Create an S3 client
    s3 = boto3.client('s3')
    if not os.path.exists("./trained_models/"):
        os.mkdir("./trained_models/")

        try:
            # Download the file
            s3.download_file(bucket_name, object_key, local_file_path)
            print(f"Downloaded {object_key} from {bucket_name} to {local_file_path}")
        except NoCredentialsError:
            print("Credentials not available or incorrect.")

# Replace these values with your own
bucket_name = 'your-s3-bucket-name'
object_key = 'path/to/your/file.txt'
local_file_path = 'local/file/path.txt'

download_from_s3(bucket_name, object_key, local_file_path)